In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
from sklearn.neural_network import MLPClassifier

import warnings
warnings.filterwarnings('ignore')


In [26]:
# Read in data
data = Path("Resources/spotify_raw_data.csv")
df = pd.read_csv(data)
df.head()

Unnamed: 0       artist                      track  danceability  energy  \
0           0   Jonas Blue            Mama - Acoustic         0.564   0.364   
1           1     Jooyoung                Same as you         0.701   0.519   
2           2    Nightwish  The Poet And The Pendulum         0.309   0.740   
3           3   Ed Sheeran                 Afire Love         0.552   0.637   
4           4  Linkin Park                   Given Up         0.655   0.885   

   key  loudness  mode  speechiness  acousticness  ...  valence    tempo  \
0   10    -5.845     0       0.0631       0.75900  ...    0.591   86.538   
1    1    -6.382     1       0.0516       0.31400  ...    0.498   89.977   
2    7    -5.917     0       0.0456       0.00854  ...    0.166  144.861   
3    5    -6.568     1       0.0445       0.46400  ...    0.333   97.970   
4    7    -4.116     1       0.0438       0.00117  ...    0.938  100.088   

             type                      id  \
0  audio_features  3VOtM6x5KXZuxSzHaaf9oO   
1  audio_features  0wpskpgrWHEDWjPgkddspN   
2  audio_features  2XD4JzbZWRuRKzzCi5Cocm   
3  audio_features  12B0Twa9kRz45Xw2PgXHJm   
4  audio_features  1fLlRApgzxWweF1JTf8yM5   

                                    uri  \
0  spotify:track:3VOtM6x5KXZuxSzHaaf9oO   
1  spotify:track:0wpskpgrWHEDWjPgkddspN   
2  spotify:track:2XD4JzbZWRuRKzzCi5Cocm   
3  spotify:track:12B0Twa9kRz45Xw2PgXHJm   
4  spotify:track:1fLlRApgzxWweF1JTf8yM5   

                                          track_href  \
0  https://api.spotify.com/v1/tracks/3VOtM6x5KXZu...   
1  https://api.spotify.com/v1/tracks/0wpskpgrWHED...   
2  https://api.spotify.com/v1/tracks/2XD4JzbZWRuR...   
3  https://api.spotify.com/v1/tracks/12B0Twa9kRz4...   
4  https://api.spotify.com/v1/tracks/1fLlRApgzxWw...   

                                        analysis_url duration_ms  \
0  https://api.spotify.com/v1/audio-analysis/3VOt...      192277   
1  https://api.spotify.com/v1/audio-analysis/0wps...      223044   
2  https://api.spotify.com/v1/audio-analysis/2XD4...      834720   
3  https://api.spotify.com/v1/audio-analysis/12B0...      314280   
4  https://api.spotify.com/v1/audio-analysis/1fLl...      189293   

  time_signature  target  
0              4       1  
1              4       1  
2              4       1  
3              4       1  
4              4       1  

[5 rows x 22 columns]

In [27]:
df.columns

Index(['Unnamed: 0', 'artist', 'track', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href',
       'analysis_url', 'duration_ms', 'time_signature', 'target'],
      dtype='object')

In [28]:
raw_data = df.drop(['Unnamed: 0',"artist","track","type","id","uri","track_href","analysis_url","time_signature"],axis=1)
raw_data.head(20)

danceability  energy  key  loudness  mode  speechiness  acousticness  \
0          0.564   0.364   10    -5.845     0       0.0631      0.759000   
1          0.701   0.519    1    -6.382     1       0.0516      0.314000   
2          0.309   0.740    7    -5.917     0       0.0456      0.008540   
3          0.552   0.637    5    -6.568     1       0.0445      0.464000   
4          0.655   0.885    7    -4.116     1       0.0438      0.001170   
5          0.564   0.534    4    -6.050     1       0.0238      0.280000   
6          0.754   0.613    5    -6.342     1       0.2290      0.318000   
7          0.414   0.708    0    -4.181     0       0.0912      0.031600   
8          0.749   0.955    7    -1.767     1       0.0867      0.054700   
9          0.477   0.946    5    -6.734     1       0.0423      0.000061   
10         0.493   0.865    7    -4.710     0       0.0357      0.286000   
11         0.667   0.361    1    -8.690     0       0.0273      0.575000   
12         0.592   0.842    1    -6.421     0       0.1100      0.066100   
13         0.624   0.857    0    -4.444     1       0.0585      0.001850   
14         0.380   0.438    5    -5.526     0       0.0962      0.025900   
15         0.628   0.485    0    -6.020     1       0.0733      0.010100   
16         0.439   0.951    3    -3.690     1       0.0569      0.001160   
17         0.394   0.971    2    -3.104     0       0.0588      0.001150   
18         0.502   0.954    5    -3.860     0       0.1110      0.022300   
19         0.341   0.867    7    -4.727     0       0.0492      0.293000   

    instrumentalness  liveness  valence    tempo  duration_ms  target  
0           0.000000    0.0839    0.591   86.538       192277       1  
1           0.000000    0.2070    0.498   89.977       223044       1  
2           0.025800    0.1190    0.166  144.861       834720       1  
3           0.000016    0.1360    0.333   97.970       314280       1  
4           0.000473    0.0448    0.938  100.088       189293       1  
5           0.000000    0.1210    0.409   85.964       206533       1  
6           0.000000    0.3350    0.904  113.196       208400       1  
7           0.000000    0.0623    0.392  199.900       240039       1  
8           0.000000    0.1120    0.761  111.960       186875       1  
9           0.027100    0.3330    0.563   79.948       233800       1  
10          0.000000    0.1520    0.192   90.979       242440       1  
11          0.005060    0.0805    0.290  134.018       242000       1  
12          0.000000    0.3130    0.708  121.077       216756       1  
13          0.003580    0.2450    0.435  104.977       200880       1  
14          0.000013    0.1540    0.349  179.404       203813       1  
15          0.000004    0.1130    0.540  119.879       237560       1  
16          0.899000    0.0926    0.247   99.533       280547       1  
17          0.540000    0.1650    0.335  138.702       235867       1  
18          0.000186    0.1350    0.199  126.934       335680       1  
19          0.000000    0.2570    0.200  182.023       240440       1

In [29]:
# Define the features X set and the target y vector
X = raw_data.drop("target",axis=1)
y = raw_data.loc[:, ["target"]]

X_scaler = StandardScaler()
X = X_scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [30]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((82, 12), (28, 12), (82, 1), (28, 1))

In [34]:

log_model = LogisticRegression()
svm_model = SVC()
mlp_model = MLPClassifier()
dec_model = tree.DecisionTreeClassifier()


In [35]:
log_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)
mlp_model.fit(X_train, y_train)
dec_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [38]:
log_acc = log_model.score(X_test,y_test)
svm_acc = svm_model.score(X_test,y_test)
mlp_acc = mlp_model.score(X_test,y_test)
dec_acc = dec_model.score(X_test,y_test)


In [44]:
print("Logistic Regression Accuracy: ", round(log_acc,2))
print("Support Vector Machine Accuracy: ", round(svm_acc,2))
print("Neural Network Accuracy: ", round(mlp_acc,2))
print("Decision Tree Accuracy: ", round(dec_acc,2))

Logistic Regression Accuracy:  0.86
Support Vector Machine Accuracy:  0.93
Neural Network Accuracy:  0.96
Decision Tree Accuracy:  0.75
